In [17]:
library(tm)
library(textstem)

Loading required package: koRpus.lang.en

Loading required package: koRpus



Loading required package: sylly

For information on available language packages for 'koRpus', run

  available.koRpus.lang()

and see ?install.koRpus.lang()



Attaching package: ‘koRpus’


The following object is masked from ‘package:tm’:

    readTagged




In [36]:
text <- "This is a simple example for demonstration how to remove stopwords in R in order to demonstrate or demonstrated demonstrating. run running ran runner"

corpus <- VCorpus(VectorSource(text))
corpus <- tm_map(corpus, content_transformer(tolower))
corpus <- tm_map(corpus, removePunctuation)
corpus <- tm_map(corpus, removeWords, stopwords("en"))
corpus <- tm_map(corpus, stripWhitespace)

# Lemmatize words using textstem
lemmatize_text <- function(text) {
  lemmatized <- textstem::lemmatize_words(unlist(strsplit(text, "\\s+")))
  paste(lemmatized, collapse = " ")
}

# Apply lemmatization to the corpus
corpus <- tm_map(corpus, content_transformer(lemmatize_text))

cat(sapply(corpus, content))

 simple example demonstration remove stopwords r order demonstrate demonstrate demonstrate run run run runner

In [28]:
train_multinomial_nb <- function(classes, docs, labels) {
  # Extract unique vocabulary from documents
  vocab <- unique(unlist(strsplit(docs, "\\s+")))
  
  # Count total number of documents
  num_docs <- length(docs)
  
  prior <- list()
  condprob <- list()
  
  for (class in classes) {
    # Count the number of documents in each class
    num_docs_in_class <- sum(labels == class)
    
    # Calculate prior probability for each class
    prior[[class]] <- num_docs_in_class / num_docs
    
    # Concatenate all text in the class
    text_class <- paste(docs[labels == class], collapse = " ")
    
    for (term in vocab) {
      # Initialize condprob for the term if it doesn't exist
      if (is.null(condprob[[term]])) condprob[[term]] <- list()
      
      # Count occurrences of term in class text
      term_count <- sum(unlist(strsplit(text_class, "\\s+")) == term)
      
      # Calculate conditional probability with Laplace smoothing
      condprob[[term]][[class]] <- (term_count + 1) / (sum(sapply(vocab, function(t) sum(unlist(strsplit(text_class, "\\s+")) == t) + 1)))
    }
  }
  
  return(list(vocab = vocab, prior = prior, condprob = condprob))
}

In [29]:
apply_multinomial_nb <- function(classes, vocab, prior, condprob, doc) {
  # Extract tokens from the document that are in the vocabulary
  tokens <- intersect(unlist(strsplit(doc, "\\s+")), vocab)
  
  scores <- list()
  
  for (class in classes) {
    # Initialize score for each class with log of prior probability
    scores[[class]] <- log(prior[[class]])
    
    for (term in tokens) {
      # Add log of conditional probability to score
      if (!is.null(condprob[[term]][[class]])) {
        scores[[class]] <- scores[[class]] + log(condprob[[term]][[class]])
      }
    }
  }
  
  # Return class with the highest score
  return(names(scores)[which.max(unlist(scores))])
}

In [34]:
# Sample dataset
docs <- c("this is a sample document", "another example document", "sample text for classification")
labels <- c("A", "B", "A")
classes <- unique(labels)

# Train the model
model <- train_multinomial_nb(classes, docs, labels)

# Apply the model to a new document
new_doc <- "this is a new document for classification"
predicted_class <- apply_multinomial_nb(classes, model$vocab, model$prior, model$condprob, new_doc)
print(predicted_class)

[1] "A"
